<a href="https://colab.research.google.com/github/abhinavbammidi1401/LLM/blob/main/Covi_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing libraries

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install datasets
!pip install -U pip setuptools wheel
!pip install -U spacy[cuda110]
!pip install -U scikit-learn
!pip install matplotlib
!pip install wikipedia
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
  Using cached setuptools-72.2.0-py3-none-any.whl.metadata (6.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
Using cached setuptools-72.2.0-py3-none-any.whl (2.3 MB)
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3061, in _parsed_pkg_info
    return self._pkg_info
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
Attribute

# Download a BERT model and its WordPiece tokenizer

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
model_name = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForMaskedLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Tokenize a phrase about COVID

In [3]:
text = "COVID-19 affects different people in different ways. Most infected people will develop mild to moderate illness and recover without hospitalization."

In [5]:
# tokenization of the text
tokens = tokenizer.tokenize(text)
print(tokens)

['co', '##vid', '-', '19', 'affects', 'different', 'people', 'in', 'different', 'ways', '.', 'most', 'infected', 'people', 'will', 'develop', 'mild', 'to', 'moderate', 'illness', 'and', 'recover', 'without', 'hospital', '##ization', '.']


In [6]:
# back to text
tokenizer.decode(tokenizer.encode(text), skip_special_tokens=True)

'covid - 19 affects different people in different ways. most infected people will develop mild to moderate illness and recover without hospitalization.'

In [7]:
print(tokenizer.tokenize('COVID'))
print(tokenizer.tokenize('hospitalization'))

['co', '##vid']
['hospital', '##ization']


**We can notice that the BERT WordPiece tokenizer (from the bert-base-cased model) tokenize the words COVID and hospitalization with subwords because they do not exist as words in the tokenizer vocabulary.**

In [8]:
# Verify that the words COVID and hospitalization DO NOT belong to the tokenizer vocabulary
vocab = [tok for tok,index in tokenizer.get_vocab().items()]
"COVID" in vocab, "hospitalization" in vocab

(False, False)

# [First test] Add 2 new tokens (whole words) into the tokenizer vocab

In [9]:
new_tokens = ['COVID', 'hospitalization']

In [10]:
print("[ BEFORE ] tokenizer vocab size:", len(tokenizer))
added_tokens = tokenizer.add_tokens(new_tokens)

print("[ AFTER ] tokenizer vocab size:", len(tokenizer))
print()
print('added_tokens:',added_tokens)
print()

# resize the embeddings matrix of the model
model.resize_token_embeddings(len(tokenizer))

[ BEFORE ] tokenizer vocab size: 30522
[ AFTER ] tokenizer vocab size: 30524

added_tokens: 2



Embedding(30524, 768)

In [11]:
# Verify that the words COVID and hospitalization DO belong to the tokenizer vocabulary
vocab = [tok for tok,index in tokenizer.get_vocab().items()]
"COVID" in vocab, "hospitalization" in vocab

(True, True)

In [12]:
tokenizer_exBERT = tokenizer

In [13]:
# tokenization of the text
tokens = tokenizer_exBERT.tokenize(text)
print(tokens)

['covid', '-', '19', 'affects', 'different', 'people', 'in', 'different', 'ways', '.', 'most', 'infected', 'people', 'will', 'develop', 'mild', 'to', 'moderate', 'illness', 'and', 'recover', 'without', 'hospitalization', '.']


In [14]:
# back to text
tokenizer_exBERT.decode(tokenizer_exBERT.encode(text), skip_special_tokens=True)

'COVID - 19 affects different people in different ways. most infected people will develop mild to moderate illness and recover without hospitalization.'

**The tokenizer with the 2 new tokens succeeded in tokenizing the words COVID and hospitalization without subwords as they belong now to the vocabulary tokenizer.**

In [15]:
# tokenization of the words COVID and hospitalization
print(tokenizer_exBERT.tokenize('COVID'))
print(tokenizer_exBERT.tokenize('hospitalization'))

['covid']
['hospitalization']


# [Second test] Add more new tokens (subwords and words) into the tokenizer vocab

What if we want to detect the whole vocabulary of a specialized corpus (and not only 2 words) in order to add it to an existing corpus?

Let's use a WordpIece tokenizer for this! (Why a WordPiece tokenizer? This is our first guess: since the BERT tokenizer is a WordPiece tokenizer, let's use a tokenizer of the same type)

1) Import pages about COVID from English Wikipedia

In [17]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=ff37626fa6ea8cce68b7a30fbd89f726ec9c4f9224f66ccd21ce7be666f9a71b
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [18]:
import wikipedia

# let's choose 2 Wikipedia pages for our demonstration (we could have choosen an infinity)
pages = ["COVID-19","COVID-19 pandemic"]

documents = list()
for p in pages:
  page = wikipedia.page(p)
  documents.append(page.content)
  print(page.title,page.url)

COVID-19 https://en.wikipedia.org/wiki/COVID-19
COVID-19 pandemic https://en.wikipedia.org/wiki/COVID-19_pandemic


2) Train a WordPiece tokenizer on the imported Wikipedia pages

In [19]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase, NFD, StripAccents
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing
from tokenizers.trainers import WordPieceTrainer

# Initialize the tokenizer with the WordPiece model
bert_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

# Apply normalizers
bert_tokenizer.normalizer = normalizers.Sequence([NFD(), Lowercase(), StripAccents()])

# Apply pre-tokenizer
bert_tokenizer.pre_tokenizer = Whitespace()

# Define the post-processing template
bert_tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B [SEP]",  # Correct template for pair
    special_tokens=[
        ("[CLS]", 1),
        ("[SEP]", 2),
    ],
)

# Instantiate a trainer for the WordPiece model
trainer = WordPieceTrainer(
    vocab_size=30522,
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

# Assuming 'documents' is your iterator over the training data
files = documents
bert_tokenizer.train_from_iterator(files, trainer)

3) Get the vocabulary that is not in the original BERT tokenizer

This step is not necessary, as the `tokenizer.add_tokens()` method will add new tokens only if they do not belong to the existing tokenizer vocabulary. However, it helps us to see what these new tokens are.

In [20]:
old_vocab = [k for k,v in tokenizer.get_vocab().items()]
new_vocab = [k for k,v in bert_tokenizer.get_vocab().items()]
idx_old_vocab_list = list()
same_tokens_list = list()
different_tokens_list = list()

for idx_new,w in enumerate(new_vocab):
  try:
    idx_old = old_vocab.index(w)
  except:
    idx_old = -1
  if idx_old>=0:
      idx_old_vocab_list.append(idx_old)
      same_tokens_list.append((w,idx_new))
  else:
      different_tokens_list.append((w,idx_new))

In [21]:
len(same_tokens_list),len(different_tokens_list),len(same_tokens_list)+len(different_tokens_list)

(4718, 2995, 7713)

We found 2995 tokens (subwords or words) that are not in the vocabulary of the original tokenizer.

In [22]:
# get list of new tokens
new_tokens = [k for k,v in different_tokens_list]
len(new_tokens), new_tokens[:10]

(2995,
 ['##oard',
  'ventilator',
  'thous',
  '##ucleated',
  'steat',
  'cle',
  'meas',
  'implic',
  'exce',
  '##ygen'])

4) Add the new tokens (subwords and words) in the vocabulary of the original BERT tokenizer

In [23]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
model_name = "bert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForMaskedLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
print("[ BEFORE ] tokenizer vocab size:", len(tokenizer))
added_tokens = tokenizer.add_tokens(new_tokens)

print("[ AFTER ] tokenizer vocab size:", len(tokenizer))
print()
print('added_tokens:',added_tokens)
print()

# resize the embeddings matrix of the model
model.resize_token_embeddings(len(tokenizer))

[ BEFORE ] tokenizer vocab size: 28996
[ AFTER ] tokenizer vocab size: 31894

added_tokens: 2995



Embedding(31894, 768)

In [25]:
# Verify if  the words COVID and hospitalization belong or not to the tokenizer vocabulary
vocab = [tok for tok,index in tokenizer.get_vocab().items()]
"COVID" in vocab, "hospitalization" in vocab

(False, False)

Let's call tokenizer_exBERT our tokenizer with the new tokens.

In [26]:
tokenizer_exBERT = tokenizer

In [27]:
# tokenization of the text
tokens = tokenizer_exBERT.tokenize(text)
print(tokens)

['CO', '##VI', '##D', '-', '19', 'affec', 't', '##s', 'diff', 'er', '##ent', 'p', '##eo', 'ple', 'in', 'diff', 'er', '##ent', 'ways', '.', 'Most', 'infecte', 'd', 'p', '##eo', 'ple', 'will', 'de', '##ve', 'lop', 'mild', 'to', 'moderate', 'ill', '##n', 'ess', 'and', 're', 'cov', 'er', 'without', 'hospit', 'al', '##ization', '.']


In [28]:
# back to text
tokenizer_exBERT.decode(tokenizer_exBERT.encode(text), skip_special_tokens=True)

'COVID - 19 affec ts diff erent peo ple in diff erent ways. Most infecte d peo ple will deve lop mild to moderate illn ess and re cov er without hospit alization.'

**As the words COVID and hospitalization do not belong to the tokenizer vocabulary, they continue to be tokenized with subwords. That's right.**

**However, only the word COVID is well tokenized. The word hospitalization is tokenized with subwords that do not start with ##. But except the first token, all other subword tokens have started with ##!**

**And we can see that many other words in the sentence are not well tokenized too.**

In [29]:
# tokenization of the words COVID and hospitalization
print(tokenizer_exBERT.tokenize('COVID'))
print(tokenizer_exBERT.tokenize('hospitalization'))

['CO', '##VI', '##D']
['hospit', 'al', '##ization']


5) Add only the new tokens that do not start with ## in the vocabulary of the original BERT tokenizer.

We know that a subword is not just a token that starts with ##, but let's see what happens if we remove all those subsowrds from the list of the new tokens.

In [30]:
# get list of new tokens as whole words
new_tokens = [tok for tok in new_tokens if tok.startswith("#") == False]
len(new_tokens), new_tokens[:10]

(1985,
 ['ventilator',
  'thous',
  'steat',
  'cle',
  'meas',
  'implic',
  'exce',
  'surfactants',
  'splitter',
  'wile'])

In [31]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
model_name = "bert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [32]:
print("[ BEFORE ] tokenizer vocab size:", len(tokenizer))
added_tokens = tokenizer.add_tokens(new_tokens)

print("[ AFTER ] tokenizer vocab size:", len(tokenizer))
print()
print('added_tokens:',added_tokens)
print()

# resize the embeddings matrix of the model
model.resize_token_embeddings(len(tokenizer))

[ BEFORE ] tokenizer vocab size: 28996
[ AFTER ] tokenizer vocab size: 30981

added_tokens: 1985



Embedding(30981, 768)

Let's call tokenizer_exBERT our tokenizer with the new tokens.

In [33]:
tokenizer_exBERT = tokenizer

In [34]:
# tokenization of the text
tokens = tokenizer_exBERT.tokenize(text)
print(tokens)

['CO', '##VI', '##D', '-', '19', 'affec', 't', '##s', 'diff', 'er', '##ent', 'p', '##eo', 'ple', 'in', 'diff', 'er', '##ent', 'ways', '.', 'Most', 'infecte', 'd', 'p', '##eo', 'ple', 'will', 'de', '##ve', 'lop', 'mild', 'to', 'moderate', 'ill', '##n', 'ess', 'and', 're', 'cov', 'er', 'without', 'hospit', 'al', '##ization', '.']


In [35]:
# back to text
tokenizer_exBERT.decode(tokenizer_exBERT.encode(text), skip_special_tokens=True)

'COVID - 19 affec ts diff erent peo ple in diff erent ways. Most infecte d peo ple will deve lop mild to moderate illn ess and re cov er without hospit alization.'

**The tokenizer continues to fail!**

**It means that we must improve the new tokens list by taking our as well the subwords that begin a word (i.e., they don't start by ##)**

In [36]:
# tokenization of the words COVID and hospitalization
print(tokenizer_exBERT.tokenize('COVID'))
print(tokenizer_exBERT.tokenize('hospitalization'))

['CO', '##VI', '##D']
['hospit', 'al', '##ization']


# [Third test] Add new tokens (only words, not subwords) into the tokenizer vocab

Let's add only the new tokens that are words, not subwords (that do not start with ## or do not are followed by a subword with ##) in the vocabulary of the original BERT tokenizer.

1) Let's use the word tokenizer (spacy) to find the most frequent words of our corpus by using scikit-learn

**Yes, but how?** Let's use a **words tokenizer like spacy** to find the most frequent words of our corpus instead of a WordPiece tokenizer which generates subwords as well.

**Observation**: here, the expression `most frequent words` means the tokens present in most of the documents.

In [37]:
import spacy
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt

In [38]:
# initialize our tokenizer with the English spaCY one
nlp = spacy.load("en_core_web_sm", exclude=['morphologizer', 'parser', 'ner', 'attribute_ruler', 'lemmatizer'])

In [39]:
def spacy_tokenizer(document, nlp=nlp):
    # tokenize the document with spaCY
    doc = nlp(document)
    # Remove stop words and punctuation symbols
    tokens = [
        token.text for token in doc if (
        token.is_stop == False and \
        token.is_punct == False and \
        token.text.strip() != '' and \
        token.text.find("\n") == -1)]
    return tokens

def dfreq(idf, N):
    return (1+N) / np.exp(idf - 1) - 1

In [40]:
%%time
# https://scikit-learn.org/stable/modules/feature_extraction.html#tfidf-term-weighting
tfidf_vectorizer = TfidfVectorizer(lowercase=False, tokenizer=spacy_tokenizer,
                                   norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)
# parse matrix of tfidf
docs = documents
length = len(docs)
result = tfidf_vectorizer.fit_transform(docs)
# print(result.shape)

# idf
idf = tfidf_vectorizer.idf_

# sorted idf, tokens and docs frequencies
idf_sorted_indexes = sorted(range(len(idf)), key=lambda k: idf[k])
idf_sorted = idf[idf_sorted_indexes]
tokens_by_df = np.array(tfidf_vectorizer.get_feature_names_out())[idf_sorted_indexes]
dfreqs_sorted = dfreq(idf_sorted, length).astype(np.int32)
tokens_dfreqs = {tok:dfreq for tok, dfreq in zip(tokens_by_df,dfreqs_sorted)}
tokens_pct_list = [int(round(dfreq/length*100,2)) for token,dfreq in tokens_dfreqs.items()]

CPU times: user 2.31 s, sys: 180 ms, total: 2.49 s
Wall time: 2.79 s


In [41]:
# we have only 2 documents (that's why we range the intervale [1,101] with a step of 50)
number_tokens_with_DF_above_pct = list()
for pct in range(1,101,50):
    index_max = len(np.array(tokens_pct_list)[np.array(tokens_pct_list)>=pct])
    number_tokens_with_DF_above_pct.append(index_max)

In [42]:
# DF = Document Frequency

# df_docfreqs = pd.DataFrame(number_tokens_with_DF_above_pct, columns=['number of tokens with DF above x%'])
# df_docfreqs.index += 1
# df_docfreqs.transpose()

# plt.plot(number_tokens_with_DF_above_pct)
# plt.title(f'Document Frequency above of {pct}%')
# plt.show()

df_docfreqs = pd.DataFrame({'pct':list(range(1,101,50)),'number of tokens with DF above pct%':number_tokens_with_DF_above_pct})
df_docfreqs.transpose()

,0,1
pct,1,51
number of tokens with DF above pct%,4366,1146


**There are 4366 words which appear in one or two documents from our 2 documents list, and 1146 which are in the documents.**

**Let's consider that the 4366 words are all important and relevant to our COVID corpus.**

**Observation**: Within a corpus with more documents, we could have used another rule as for example: keeping only words which are at least in 10% of the documents list.

**Get the vocabulary that is not in the original BERT tokenizer**

This step is not necessary, as the `tokenizer.add_tokens()` method will add new tokens only if they do not belong to the existing tokenizer vocabulary. However, it helps us to see what these tokens are.

In [43]:
# list of new tokens
pct = 1
index_max = len(np.array(tokens_pct_list)[np.array(tokens_pct_list)>=pct])
new_tokens = tokens_by_df[:index_max]
# print(len(new_tokens))

old_vocab = [k for k,v in tokenizer.get_vocab().items()]
new_vocab = [token for token in new_tokens]
idx_old_vocab_list = list()
same_tokens_list = list()
different_tokens_list = list()

for idx_new,w in enumerate(new_vocab):
  try:
    idx_old = old_vocab.index(w)
  except:
    idx_old = -1
  if idx_old>=0:
      idx_old_vocab_list.append(idx_old)
      same_tokens_list.append((w,idx_new))
  else:
      different_tokens_list.append((w,idx_new))

In [44]:
len(same_tokens_list),len(different_tokens_list),len(same_tokens_list)+len(different_tokens_list)

(3660, 706, 4366)

**We found 706 tokens (whole words) that are not in the vocabulary of the original tokenizer and the words COVID and hospitalization belong to the new tokens list.**

In [45]:
# get list of new tokens
new_tokens = [k for k,v in different_tokens_list]
print(len(new_tokens), new_tokens[:20])

706 ['0.002', '0.01', '0.1', '0.4', '0.5', '1.02', '1.4', '1.7', '2021', '266', '3–4', '4.6', '50,000', '6,174', '7.4', 'Adhanom', 'B.1.1.529', 'BMJ', 'CDC', 'CFR']


In [46]:
"COVID" in new_tokens, "hospitalization" in new_tokens

(True, True)

Add new tokens (only whole words, not subwords) in the vocabulary of the original BERT tokenizer

In [47]:
# import model and tokenizer
from transformers import AutoModelForMaskedLM, AutoTokenizer
model_name = "bert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [48]:
print("[ BEFORE ] tokenizer vocab size:", len(tokenizer))
added_tokens = tokenizer.add_tokens(new_tokens)

print("[ AFTER ] tokenizer vocab size:", len(tokenizer))
print()
print('added_tokens:',added_tokens)
print()

# resize the embeddings matrix of the model
model.resize_token_embeddings(len(tokenizer))

[ BEFORE ] tokenizer vocab size: 28996
[ AFTER ] tokenizer vocab size: 29702

added_tokens: 706



Embedding(29702, 768)

Let's call tokenizer_exBERT our tokenizer with the new tokens.

In [49]:
tokenizer_exBERT = tokenizer

In [50]:
# tokenization of the text
tokens = tokenizer_exBERT.tokenize(text)
print(tokens)

['COVID-19', 'affects', 'different', 'people', 'in', 'different', 'ways', '.', 'Most', 'infected', 'people', 'will', 'develop', 'mild', 'to', 'moderate', 'illness', 'and', 'recover', 'without', 'hospitalization', '.']


In [51]:
# back to text
tokenizer_exBERT.decode(tokenizer_exBERT.encode(text), skip_special_tokens=True)

'COVID-19 affects different people in different ways. Most infected people will develop mild to moderate illness and recover without hospitalization.'

**The tokenizer with the new tokens (only whole words) did succeed in tokenizing the words, COVID and hospitalization (and not only these ones, all of them!)**

**It means that is fundamental to add new tokens that are only whole words to an existing subword tokenizer like WordPiece and not subwords!**

In [52]:
# tokenization of the words COVID and hospitalization
print(tokenizer_exBERT.tokenize('COVID'))
print(tokenizer_exBERT.tokenize('hospitalization'))

['COVID']
['hospitalization']


# Let's check the impact of our enriched tokenizer

Let's use a text about COVID taken from a newspaper site (not from Wikipedia)

In [53]:
# source: https://edition.cnn.com/2021/04/05/health/us-coronavirus-monday/index.html
text = 'Experts say Covid-19 vaccinations in the US are going extremely well -- but not enough people are protected yet and the country may be at the start of another surge. \
The US reported a record over the weekend with more than 4 million Covid-19 vaccine doses administered in 24 hours, according to the Centers for Disease Control and Prevention. \
And the country now averages more than 3 million doses daily, according to CDC data. \
But only about 18.5% of Americans are fully vaccinated, CDC data shows, and Covid-19 cases in the country have recently seen concerning increases. \
"I do think we still have a few more rough weeks ahead," Dr. Celine Gounder, an infectious diseases specialist and epidemiologist, told CNN on Sunday. \
"What we know from the past year of the pandemic is that we tend to trend about three to four weeks behind Europe in terms of our pandemic patterns."'

In [54]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
model_name = "bert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [55]:
tokens = tokenizer.tokenize(text)
print('number of tokens by the original BERT tokenizer:', len(tokens))

tokens = tokenizer_exBERT.tokenize(text)
print('number of tokens by the enriched tokenizer:', len(tokens))

number of tokens by the original BERT tokenizer: 203
number of tokens by the enriched tokenizer: 188


**As expected, we find that the enriched tokenizer needs less tokens (almost 30%) to tokenize the text on COVID than the original BERT tokenizer.**